In [124]:
import pandas as pd
import numpy as np
import timeit
import operator
from __future__ import division # for having results of accuracy in floating numbers
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

In [125]:
def transformData(df):
    temp=[]
    for row in df.iterrows():
        index, data = row
        temp.append(data.tolist())
    return temp

columns = ['user','movie','rating','idk']
train_90 = pd.read_csv("ratings_training_90.csv",names = columns)
test_10 = pd.read_csv("ratings_test_10.csv",names = columns)
test = transformData(test_10)

In [126]:
df_train= train_90.pivot(index='user', columns='movie', values='rating')
df_test= test_10.pivot(index='user', columns='movie', values='rating')
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
time = []
rmsT = []

In [127]:
def getNeighbors(trainingSet, testInstance, k):
    testuser = int(testInstance[0])  
    testmovie = testInstance[1] 
    test_user_rating = testInstance[2]
    reduced_Data = trainingSet[trainingSet[testmovie] > 0]   # Subset of data for test movie id
    Other_users_rated_as = np.array(pd.DataFrame(reduced_Data)[int(testmovie)])
    filtered_users = reduced_Data.index.values   # users who have watched the movie being rated 
    pred_user = trainingSet[trainingSet.index == testuser]
    distances = []     # calculating distance
    d = 0 
    for i in filtered_users:
        dist = distance.euclidean(reduced_Data[reduced_Data.index == i],pred_user)
        distances.append([testuser,i,int(reduced_Data[reduced_Data.index==i][testmovie].values),dist])
    distances.sort(key=operator.itemgetter(3))
    check = len(distances)
    neighbors = []
    if (check > k):
        for i in range(k):
            neighbors.append(distances[i])
    else:
        for i in range(check):
            neighbors.append(distances[i])
    return neighbors

In [128]:
def get(neighbor):
    num_of_neighbors = len(neighbor)
    onlyDistances = []
    onlyRaings = []
    Di = []
    Winum = []
    Widen = []
    Di_rating = []
    cl = 0
    for x in range(num_of_neighbors):
        onlyDistances.append(neighbor[x][3])
        onlyRaings.append(neighbor[x][2])
    Dmax = max(onlyDistances)
    Dmin = min(onlyDistances)
    if (Dmax - Dmin != 0):
        for x in range(num_of_neighbors):
            #print(Dmax,neighbor[x][3],Dmin)
            Winum.append((Dmax-neighbor[x][3])/(Dmax - Dmin)) 
            Di.append(Winum[x]*onlyDistances[x])
            Di_rating.append(Di[x] * onlyRaings[x])
        prediction = round(sum(Di_rating)/sum(Di),1)
    else:
        for x in range(len(neighbor)):
            cl += neighbor[x][2]
        prediction = round(cl/float(len(neighbor)), 1)    
    #return(Winum,Di,Di_rating,prediction)
    return(prediction)

In [139]:
a = int(len(test))
start = timeit.default_timer()
pred = []
predictions=[]
k = 10
#a = 20
for x in range(a):
    neigh = getNeighbors(df_train, test[x], k)
    result = get(neigh)
    predictions.append([test[x][0],test[x][1],test[x][2],result])
    pred.append(result)
stop = timeit.default_timer()
stop = stop - start

In [140]:
# Weighted KNN
# time.  K = 3:   660.6614640900079  RMSE: 1.2707091613268062
# time.  K = 5:   662.0650260520051  RMSE: 1.1444297704440511
# time.  K = 10:  660.425073802995  RMSE: 1.054834443814851

In [141]:
pred = pd.DataFrame(predictions)
pred.to_csv('ratings_predictions_10_knn_wt_k10.csv')

In [142]:
time.append(stop)
rms = sqrt(mse(pred[2],pred[3]))
rmsT.append(rms)

In [143]:
time, rmsT

([660.6614640900079, 662.0650260520051, 660.425073802995],
 [1.2707091613268062, 1.1444297704440511, 1.054834443814851])